<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/doc/docs/openhosta_phi4_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenHosta basic example with a Local Phi-4 or gpt-4o

This colab demonstrate simple use cases of OpenHosta. If you do not have an OpenAI (or other) API KEY, you can run the first part **Install a local Phi-4**. Otherwise, directly jump to step 2: **Basic Usage**.

## Install a local Phi-4

In [1]:
!apt install -y screen

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  screen
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 672 kB of archives.
After this operation, 1,029 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 screen amd64 4.9.0-1 [672 kB]
Fetched 672 kB in 1s (745 kB/s)
Selecting previously unselected package screen.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../screen_4.9.0-1_amd64.deb ...
Unpacking screen (4.9.0-1) ...
Setting up screen (4.9.0-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
#!curl -fsSL https://ollama.com/install.sh | sh

In [7]:
# Load and run the model:
!screen -dmS ollama ollama serve

In [7]:
# Can take 5 minutes
!ollama run phi4-mini hello --verbose  2>&1 | grep -E "([^0]0%|Bonjour|:)"

total duration:       315.970426ms
load duration:        28.64048ms
prompt eval count:    4 token(s)
prompt eval duration: 8ms
prompt eval rate:     500.00 tokens/s
eval count:           10 token(s)
eval duration:        277ms
eval rate:            36.10 tokens/s


In [5]:
!pip install OpenHosta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.2 MB/s eta 0:00:00


## Basic Usage of OpenHosta

### Configure the LLM that you want to use

In [8]:
from OpenHosta import config

# If you have an OpenAI API key you can use it like this:
# Use default model (gpt-4o) through API key
# config.set_default_apiKey(<<YOUR API KEY>>)

# Comment this if you use OpenAI models
# Use Microsoft local Phi-4 through ollama
my_model=config.Model(
    base_url="http://localhost:11434/v1/chat/completions",
    model="phi4-mini", api_key="none", timeout=120
)
config.set_default_model(my_model)


### Emulate functions using the seleted LLM

In [14]:
from OpenHosta import emulate_async

In [21]:
async def translate(text:str, language:str)->str:
    """
    This function translates the text in the “text” parameter into the language specified in the “language” parameter.
    """
    return await emulate_async()

result = await translate("Hello World!", "French")
print(result)

{"type":"string","content":"Bonjour le Monde !"}


In [ ]:
# You can select another model like this
#my_model = config.Model(
#    model="gpt-4o-mini",
#    base_url="https://api.openai.com/v1/chat/completions",
#    api_key=<<API KEY>>
#)

In [17]:
async def find_name_age(sentence:str, id:dict)->dict:
    """
    This function find in a text the name and the age of a personn.

    Args:
        sentence: The text in which to search for information
        id: The dictionary to fill in with information

    Return:
        A dictionary identical to the one passed in parameter, but filled with the information.
        If the information is not found, fill with the values with “None”.
    """
    return await emulate_async(model=my_model)

return_dict = {"name": "", "age": ""}
result = await find_name_age("Hello, I'm John Wick, i'm 30 and I live in New York", return_dict)
print(result)

{'type': 'dict', 'data': {'name': 'John Wick', 'age': '30'}}


### Specify advanced return types

In [24]:
from typing import Dict, Tuple, List

async def analyze_text(text: str) -> Dict[str, List[Tuple[int, str]]]:
    """Analyze text to map each word to a list of tuples containing word length and word."""
    return await emulate_async()

# Example usage
analysis = await analyze_text("Hello, World!")

print(analysis)
print(type(analysis))

{'type': 'object', 'wordAnalysis': [{'length': 6, 'word': ['Hello']}, {'length': 1, 'word': [',']}]}
<class 'dict'>


### Specify pydantic return strucures

OpenHosta is compatible with pydantic. You can specify pydantic input and output types and OpenHosta will propagate schema and Field documentation to the LLM.

In [25]:
!pip install pydantic

In [28]:
from pydantic import BaseModel, Field

class Personn(BaseModel):
    name: str = Field(..., description = "The full name")
    age: int

async def find_name_age_pydantic(sentence:str)->Personn:
    """
    This function find in a text the name and the age of a personn.

    Args:
        sentence: The text in which to search for information

    Return:
        A Pydantic model, but filled with the information.
        If the information is not found, fill with the values with “None”.
    """
    return await emulate_async()

result = await find_name_age_pydantic("Luke Skywalker is very surprising: he's only 27 when he becomes a Jedi.")
print(result)

{'properties': {'name': {'description': 'The full name', 'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}}, 'required': ['name', 'age'], 'title': 'Personn', 'type': 'object'}


[TypeConverter.check]: Failed to convert pydantic type from the LLM. Keep original type.


### Limitations

The emulation is limited by the LLM capabilities. Try to have it count r in strawberrry and you will get into troubles ;-).
Make sure the LLM is capable and not alucinating before you implement an emulated function.

In [29]:
async def find_occurence_of_a_word(word :str, text: str) -> int:
    """
    This function takes a word and a text and returns
    the number of times the word appears in the text.
    """
    return await emulate_async()

await find_occurence_of_a_word("Hello", "Hello World Hello!")


[TypeConverter.check]: Failed to convert basic type from the LLM. Keep original type.


{'type': 'integer', 'value': 2}